# 01 — Data Exploration & Cleaning
This notebook downloads the Fitzpatrick17k dataset, cleans it, and analyzes the class distribution.

In [ ]:
# Uncomment to install dependencies in Colab
# !pip install -q imagehash wandb google-cloud-storage google-cloud-aiplatform

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add project root to path (for Colab: mount drive or clone repo first)
# sys.path.insert(0, '/content/skin_tone_classifier')

from src.data.prepare import (
    load_metadata,
    validate_fitzpatrick_labels,
    encode_grouped_labels,
    validate_images,
    filter_human_images,
    deduplicate_images,
    compute_class_distribution,
    generate_cleaning_report,
    stratified_split,
    download_images,
)

In [ ]:
# Configuration
CSV_PATH = "data/fitzpatrick17k.csv"
IMAGE_DIR = "data/images"
OUTPUT_DIR = "data/cleaned"
RANDOM_SEED = 42

In [ ]:
# Load metadata
df = load_metadata(CSV_PATH)
print(f"Total rows: {len(df)}")
print(f"Columns: {list(df.columns)}")
df.head()

In [ ]:
# Explore raw Fitzpatrick distribution
print("\nRaw Fitzpatrick distribution:")
print(df["fitzpatrick"].value_counts().sort_index())

fig, ax = plt.subplots(figsize=(8, 5))
df["fitzpatrick"].value_counts().sort_index().plot(kind="bar", ax=ax, color="steelblue")
ax.set_title("Raw Fitzpatrick Skin Type Distribution")
ax.set_xlabel("Fitzpatrick Type")
ax.set_ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
# Download images (uncomment and run if you need to download from URLs)
# downloaded = download_images(df, IMAGE_DIR)
# print(f"Downloaded {downloaded} images")

In [ ]:
# Step 1: Validate Fitzpatrick labels
original_count = len(df)
df = validate_fitzpatrick_labels(df)
print(f"After label validation: {len(df)} ({original_count - len(df)} dropped)")

In [ ]:
# Step 2: Validate images
before = len(df)
df = validate_images(IMAGE_DIR, df)
print(f"After image validation: {len(df)} ({before - len(df)} dropped)")

In [ ]:
# Step 3: Filter to human-only images
before = len(df)
df = filter_human_images(IMAGE_DIR, df)
print(f"After human filter: {len(df)} ({before - len(df)} dropped)")

In [ ]:
# Step 4: Deduplicate
before = len(df)
df = deduplicate_images(IMAGE_DIR, df)
print(f"After deduplication: {len(df)} ({before - len(df)} dropped)")

In [ ]:
# Step 5: Encode grouped labels
df = encode_grouped_labels(df)
print("\nGrouped label distribution:")
print(df["skin_tone_group"].value_counts().sort_index())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df["fitzpatrick"].value_counts().sort_index().plot(kind="bar", ax=axes[0], color="steelblue")
axes[0].set_title("Cleaned: 6-Class Distribution")
axes[0].set_xlabel("Fitzpatrick Type")
axes[0].set_ylabel("Count")

df["skin_tone_group"].value_counts().sort_index().plot(kind="bar", ax=axes[1], color="coral")
axes[1].set_title("Cleaned: 3-Class Grouped Distribution")
axes[1].set_xlabel("Skin Tone Group")
axes[1].set_ylabel("Count")

plt.tight_layout()
plt.show()

In [ ]:
# Class distribution report
dist = compute_class_distribution(df, "skin_tone_group")
print("\nClass Distribution Report:")
for cls, info in sorted(dist.items()):
    print(f"  {cls}: {info['count']} images ({info['percentage']:.1f}%)")

imbalance_ratio = max(d["count"] for d in dist.values()) / min(d["count"] for d in dist.values())
print(f"\nImbalance ratio: {imbalance_ratio:.2f}x")

In [ ]:
# Stratified split
train_df, val_df, test_df = stratified_split(df, "skin_tone_label", (0.7, 0.15, 0.15), seed=RANDOM_SEED)

print(f"\nSplit sizes: train={len(train_df)}, val={len(val_df)}, test={len(test_df)}")

for split_name, split_df in [("Train", train_df), ("Val", val_df), ("Test", test_df)]:
    print(f"\n{split_name} distribution:")
    print(split_df["skin_tone_group"].value_counts().sort_index())

In [ ]:
# Save cleaned data
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
df.to_csv(f"{OUTPUT_DIR}/fitzpatrick17k_cleaned.csv", index=False)
train_df.to_csv(f"{OUTPUT_DIR}/train.csv", index=False)
val_df.to_csv(f"{OUTPUT_DIR}/val.csv", index=False)
test_df.to_csv(f"{OUTPUT_DIR}/test.csv", index=False)
print(f"\nSaved cleaned data to {OUTPUT_DIR}/")

In [ ]:
# Summary
print("\n" + "="*60)
print("DATA CLEANING SUMMARY")
print("="*60)
print(f"Original images:     {original_count}")
print(f"After cleaning:      {len(df)}")
print(f"Total dropped:       {original_count - len(df)}")
print(f"Imbalance ratio:     {imbalance_ratio:.2f}x")
print(f"Train/Val/Test:      {len(train_df)}/{len(val_df)}/{len(test_df)}")
print("="*60)